In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split

import re
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

import zipfile
from tqdm import tqdm
from google.colab import drive

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
stem = SnowballStemmer('english')
lemma = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Функции

#### 1 Задание

In [ ]:
def preprocess_text(text: str):
  text = (re.sub(r"([\d.,!?'/:])", "", text)).lower()
  return text

In [ ]:
def create_binominal_tensor(text, n=0):
  vocab = list(dict.fromkeys([word for word in word_tokenize(text)]))
  sentence = text.split('.')[n].split()
  return ([1 if word in sentence else 0 for word in vocab])

In [ ]:
def create_unitarian_tensor(text, n=0):
  vocab = list(dict.fromkeys([word for word in word_tokenize(text)]))
  sentence = text.split('.')[n].split()
  result = []
  for s_word in sentence:
    preresult = []
    for v_word in vocab:
      preresult.append(1) if s_word == v_word else preresult.append(0)
    result.append(preresult)
  return result

In [ ]:
def cleanalyzer(text):
    text = (re.sub(r'\d+', '', text)).lower()
    text = word_tokenize(text)
    text = [lemma.lemmatize(word) for word in text if word not in stop_words]
    text = [word for word in text if len(word) > 2 and len(word) < 30]
    text = [stem.stem(word) for word in text]
    text = ' '.join(text)
    return text

In [ ]:
def token_to_index(text, split=False):
  return {word: index for index, word in enumerate(text.split())} if split else {word: index for index, word in enumerate(text)}

In [ ]:
def index_to_token(text, split=False):
  return {index: word for index, word in enumerate(text.split())} if split else {index: word for index, word in enumerate(text).split()}

#### Общие функции

In [ ]:
def train(dataloader, model, loss_fn, optimizer, epoch):
    size = len(dataloader.dataset)
    model.train()
    loss_sum = 0

    total_samples = 0.0
    correct_samples = 0.0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X.type(torch.long))
        # torch.max(y, 1)[1]
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_sum += loss

        total_samples += y.shape[0]
        _, prediction_indices = torch.max(pred, 1)
        correct_samples += torch.sum(prediction_indices==y)

    if epoch == 0 or (epoch+1)%5 == 0:
        print(f"train loss: {loss.item():>7f}")

    return float(correct_samples) / total_samples, (loss_sum / len(dataloader)).cpu().detach().numpy()

In [ ]:
def test(dataloader, model, loss_fn, epoch):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    model.eval()

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    if epoch == 0 or (epoch+1)%5 == 0:
      print(f"test: accuracy: {(100*correct):>0.1f}%, val loss: {test_loss:>8f} \n")
    return correct, test_loss

In [ ]:
def train_test(train_dl, test_dl, model, loss_fn, optimizer, epochs=10):
  history = pd.DataFrame(columns=['epoch', 'train loss', 'train accuracy', 'val loss', 'test accuracy'])
  for epoch in range(epochs):
      if epoch == 0 or (epoch+1)%5 == 0:
        print(f"Epoch {epoch+1}:")
      train_acc, train_loss = train(train_dl, model, loss_fn, optimizer, epoch)
      test_acc, val_loss = test(test_dl, model, loss_fn, epoch)
      history.loc[epoch] = [epoch, train_loss, train_acc, val_loss, test_acc]
  return model, history

In [ ]:
def train_test_info(history):
  fig, _axs = plt.subplots(nrows=2, figsize=(10, 10))
  axs = _axs.flatten()
  axs[0].plot(history['train loss'], label='Train loss')
  axs[0].plot(history['val loss'], label='Val loss')
  axs[0].legend(frameon=False);
  axs[1].plot(history['train accuracy'], label='Train accuracy')
  axs[1].plot(history['test accuracy'], label='Test accuracy')
  axs[1].legend(frameon=False)
  plt.show()

In [ ]:
def conf_mat(model, test_dl, classes):
  model.eval()
  true = torch.empty(0).to(device)
  predict = torch.empty(0).to(device)

  with torch.no_grad():
    for inputs, targets in test_dl:
        inputs, targets = inputs.to(device), targets.to(device)
        inputs = inputs.view(inputs.shape[0], -1)

        outputs = model(inputs)

        _, predictions_indices = torch.max(outputs, 1)

        true = torch.cat((true, targets))
        predict = torch.cat((predict, predictions_indices))

  plt.figure(figsize=(15,15))
  sns.heatmap(
    pd.DataFrame(
        confusion_matrix(true.cpu().numpy().astype("int"), predict.cpu().numpy().astype("int")),
    ),
    annot=True,
    fmt="d",
    xticklabels=classes,
    yticklabels=classes
  );

  total_accuracy = torch.sum(true==predict) / true.shape[0]
  print(f"total accuracy: {total_accuracy:.3f}")

## 1. Представление и предобработка текстовых данных в виде последовательностей

1.1 Представьте первое предложение из строки `text` как последовательность из индексов слов, входящих в это предложение

In [ ]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [ ]:
text = preprocess_text(text)
create_binominal_tensor(text)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

1.2 Представьте первое предложение из строки `text` как последовательность векторов, соответствующих индексам слов. Для представления индекса в виде вектора используйте унитарное кодирование. В результате должен получиться двумерный тензор размера `количество слов в предложении` x `количество уникальных слов`

In [ ]:
for string in create_unitarian_tensor(text):
  print(string)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0,

In [ ]:
(len(create_unitarian_tensor(text)), len(create_unitarian_tensor(text)[0]))

(27, 25)

1.3 Решите задачу 1.2, используя модуль `nn.Embedding`

In [ ]:
first = str(text.split('.')[0])
first

'select your preferences and run the install command stable represents the most currently tested and supported version of pytorch note that libtorch is only available for c++'

In [ ]:
token_to_index(first, True)

{'select': 0,
 'your': 1,
 'preferences': 2,
 'and': 14,
 'run': 4,
 'the': 10,
 'install': 6,
 'command': 7,
 'stable': 8,
 'represents': 9,
 'most': 11,
 'currently': 12,
 'tested': 13,
 'supported': 15,
 'version': 16,
 'of': 17,
 'pytorch': 18,
 'note': 19,
 'that': 20,
 'libtorch': 21,
 'is': 22,
 'only': 23,
 'available': 24,
 'for': 25,
 'c++': 26}

In [ ]:
embedding = nn.Embedding(26, 1)
lookup_tensor = torch.tensor(list(token_to_index(first, True).values()), dtype=torch.long)
embedded = embedding(lookup_tensor)
embedded.shape

IndexError: ignored

In [ ]:
embedded

In [ ]:
weights = torch.zeros(embedding.weight.shape)
embedding2 = nn.Embedding(26, 1)
print(embedding2.weight)
embedding2.weight = torch.nn.Parameter(weights.view(embedding.weight.shape))
print(embedding2.weight)
embedding2(lookup_tensor)

Parameter containing:
tensor([[-0.9697],
        [ 1.3911],
        [ 0.6879],
        [ 1.1484],
        [-1.0278],
        [-0.3674],
        [-0.7074],
        [-1.1781],
        [ 0.1069],
        [ 0.2751],
        [ 2.4118],
        [-0.2766],
        [ 0.8202],
        [ 1.0380],
        [ 0.0960],
        [-0.1787],
        [-0.3483],
        [ 0.1635],
        [ 1.1771],
        [-0.0719],
        [-0.0996],
        [ 0.4208],
        [ 1.7619],
        [ 0.1411],
        [ 0.0048],
        [-0.3291]], requires_grad=True)
Parameter containing:
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], requires_grad=True)


IndexError: ignored

## 2. Классификация фамилий по национальности (ConvNet)

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`.

2.2 Закодировать национальности числами, начиная с 0.

2.3 Разбить датасет на обучающую и тестовую выборку

2.4 Реализовать класс `Vocab` (токен = __символ__)
  * добавьте в словарь специальный токен `<PAD>` с индексом 0
  * при создании словаря сохраните длину самой длинной последовательности из набора данных в виде атрибута `max_seq_len`

2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса>
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины

2.6. Обучить классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`. Рассмотрите два варианта:
    - когда токен представляется в виде унитарного вектора и модуль `nn.Embedding` не обучается
    - когда токен представляется в виде вектора небольшой размерности (меньше, чем размер словаря) и модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

In [ ]:
class Vocab:
    def __init__(self, data):
        self.idx_to_token = {0:"<PAD>"}
        self.token_to_idx = {"<PAD>":0}
        self.max_seq_len = 0
        self.gennerate_vocab(data)
        self.vocab_len = len(self.token_to_idx)

    def add_token(self, token):
        if token not in self.token_to_idx:
            self.token_to_idx[token] = len(self.token_to_idx)
            self.idx_to_token[len(self.idx_to_token)] = token

    def gennerate_vocab(self, data):
        for surname in data:
          if len(surname) > self.max_seq_len:
            self.max_seq_len = len(surname)
          for char in preprocess_text(surname):
            self.add_token(char)


In [ ]:
class SurnamesDataset():
    def __init__(self, X, y, vocab: Vocab, targets):
        self.X = X
        self.y = y
        self.vocab = vocab
        self.targets = targets

    def vectorize(self, surname):
        result = []
        for char in preprocess_text(surname):
            preresult = []
            for vocab_char in self.vocab.idx_to_token.values():
                preresult.append(1) if char == vocab_char else preresult.append(0)
            result.append(preresult)
        for i in range(self.vocab.max_seq_len - len(preprocess_text(surname))):
            result.append([1 if i==0 else 0 for i in range(self.vocab.vocab_len)])
        return torch.Tensor(result)

    def unvectorize(self, vector):
      result = ''
      for i in range(len(vector)):
        for j in range(len(vector[i])):
          if vector[i][j] == 1:
            result += vocab.idx_to_token[j]
      return result

    def __getitem__(self, idx) -> tuple:
        return self.vectorize(self.X[idx]), self.targets[idx]

    def __len__(self) -> int:
        return len(self.X)

    def y_len(self) -> int:
        return self.y.unique().shape[0]

2.1 Считать файл surnames/surnames.csv.

In [ ]:
zf = zipfile.ZipFile('drive/MyDrive/datasets/surnames.zip')
for file in tqdm(zf.infolist()):
    zf.extract(file)

100%|██████████| 2/2 [00:00<00:00, 423.75it/s]


In [ ]:
df = pd.read_csv("/content/surnames/surnames.csv")
df.head()

,surname,nationality
0,Woodford,English
1,Coté,French
2,Kore,English
3,Koury,Arabic
4,Lebzak,Russian


2.2 Закодировать национальности числами, начиная с 0.

In [ ]:
nationalities = {nationality: index for index, nationality in enumerate(df["nationality"].unique())}
df["index"] = df["nationality"].map(nationalities)
targets = torch.tensor(df["index"])
df.head()

,surname,nationality,index
0,Woodford,English,0
1,Coté,French,1
2,Kore,English,0
3,Koury,Arabic,2
4,Lebzak,Russian,3


2.4 Реализовать класс `Vocab` (токен = __символ__)
  * добавьте в словарь специальный токен `<PAD>` с индексом 0
  * при создании словаря сохраните длину самой длинной последовательности из набора данных в виде атрибута `max_seq_len`

In [ ]:
vocab = Vocab(df["surname"])
vocab.max_seq_len

17

In [ ]:
targets

tensor([ 0,  1,  0,  ..., 12,  0,  9])

In [ ]:
vocab.vocab_len

52

2.3 Разбить датасет на обучающую и тестовую выборку\
2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса>
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины


In [ ]:
dataset = SurnamesDataset(
    df["surname"].tolist(),
    torch.tensor(df["index"], dtype=torch.long),
    vocab,
    targets
)


print(f"dataset_size: {len(dataset)}")


train_size = int(len(dataset)*0.75)
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size = 64, shuffle = True, pin_memory=False)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size = 1, pin_memory = False)

classes = df["nationality"].unique()

print(f"train_size : {train_size}")
print(f"test_size : {test_size}")

dataset_size: 10980
train_size : 8235
test_size : 2745


In [ ]:
print(f'target for this vector is: {dataset[1][1]}')
for i in dataset[1][0]:
  print(i.cpu().numpy())

target for this vector is: 1
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[1. 0. 0. 0

In [ ]:
dataset.unvectorize(dataset[1][0])

'coté<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>'

2.6. Обучить классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`. Рассмотрите два варианта:
    - когда токен представляется в виде унитарного вектора и модуль `nn.Embedding` не обучается
    - когда токен представляется в виде вектора небольшой размерности (меньше, чем размер словаря) и модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Embedding(vocab.vocab_len, vocab.vocab_len),
    # torch.nn.Conv1d(52, vocab.vocab_len, vocab.max_seq_len),
    torch.nn.Conv1d(vocab.max_seq_len, 32, 3),
    torch.nn.MaxPool2d(2),
    torch.nn.LeakyReLU(0.1),
    torch.nn.Dropout(0.5),
    torch.nn.Flatten(),
    torch.nn.Linear(18, 512),
    torch.nn.Dropout(0.5),
    torch.nn.LeakyReLU(0.1),
    torch.nn.Linear(512, len(classes))
).to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
model, history = train_test(trainloader, testloader, model, criterion, optimizer, 5)

Epoch 1:
1
2
3


RuntimeError: ignored

In [ ]:
class SurnameModel(nn.Module):
    def __init__(self, embed_num, embed_dim, classes_num, seq_len, use_embed=True):
        super(SurnameModel, self).__init__()

        self.embed_dim = embed_dim
        self.use_embed = use_embed
        self.seq_len = seq_len

        if self.use_embed:
            self.embed = nn.Embedding(embed_num, embed_dim)

        self.relu = nn.ReLU()
        self.conv1 = nn.Conv1d(in_channels=embed_dim, out_channels=128, kernel_size=3)
        self.pool1 = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3)
        self.pool2 = nn.MaxPool1d(2)

        self.fc1 = nn.Linear(64, classes_num)

    def forward(self, x):
        print(11)
        if self.use_embed:
            x = self.embed(x)
        # print(x.shape)
        print(x)
        x = x.reshape(len(x), self.embed_dim, self.seq_len).type(torch.FloatTensor)
        print(x)
        x = self.pool1(self.relu(self.conv1(x)))
        print(x)
        x = self.pool2(self.relu(self.conv2(x)))
        print(15)
        x, _ = x.max(dim=-1)
        print(16)

        return self.fc1(x)

In [ ]:
model = SurnameModel(
    vocab.vocab_len, vocab.vocab_len, len(targets), vocab.max_seq_len, use_embed=False
).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
n_epochs = 5

In [ ]:
model, history = train_test(trainloader, testloader, model, criterion, optimizer, 5)

Epoch 1:
1
2
3
11
tensor([ 3,  3,  2,  2,  4, 11,  3,  0,  2,  3,  6,  7,  3, 11,  0,  6,  2,  0,
         0,  2,  9, 14,  2,  5,  6,  2,  0,  3,  0,  4,  2,  0,  0, 13,  3, 12,
         0,  2,  6,  0,  3,  2,  0,  0, 11,  0,  2,  0,  4,  4,  3,  2],
       device='cuda:0')


RuntimeError: ignored

## 3. Классификация обзоров на фильмы (ConvNet)

Датасет: https://disk.yandex.ru/d/tdinpb0nN_Dsrg

2.1 Создайте набор данных на основе файлов polarity/positive_reviews.csv (положительные отзывы) и polarity/negative_reviews.csv (отрицательные отзывы). Разбейте на обучающую и тестовую выборку.
  * токен = __слово__
  * данные для обучения в датасете представляются в виде последовательности индексов токенов
  * словарь создается на основе _только_ обучающей выборки. Для корректной обработки ситуаций, когда в тестовой выборке встретится токен, который не хранится в словаре, добавьте в словарь специальный токен `<UNK>`
  * добавьте предобработку текста

2.2. Обучите классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`
    - подберите адекватную размерность вектора эмбеддинга:
    - модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`


2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)
* Целевое значение accuracy на валидации - 70+%